# Capstone project - Amsterdam

For this project I want to investigate what the optimal location for a new hotel in Amsterdam would be. I selected 16 of the most centrally located boroughs of the city (all within the ring road) as options. There are already many hotels in Amsterdam, so I'd like to find the borough that meets the following criteria.

1- It has to be in the top 5 out of the selected boroughs in terms of variety of food & drinks venues

2- It should have multiple museums within walking distance

3- Museums should be in the top 10 of the borough

From the boroughs that meet the criteria, I will choose the one that has the lowest density of hotels.

To conduct my analysis, I will use the Foursquare API. I will request which venues are within short walking distance of the centre of the borough. I hope the data is as complete for Amsterdam as it was for Toronto and New York.


In [1]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
ams = pd.read_csv('Geo_Amsterdam.csv')
ams.head()

,Borough,Latitude,Longitude,Unnamed: 3,Unnamed: 4
0,Bos en Lommer,52.380688,4.855018,NaN,NaN
1,Staatsliedenbuurt,52.382233,4.870997,NaN,NaN
2,Jordaan,52.375251,4.877648,NaN,"52.375251, 4.877648"
3,Negen straatjes,52.370399,4.884536,NaN,NaN
4,Oud-West,52.364865,4.866000,NaN,NaN


In [3]:
# Leidseplein is the most central landmark of Amsterdam among the selected boroughs
address = 'Leidseplein, Amsterdam, the Netherlands'

geolocator = Nominatim(user_agent="ams_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Leidsplein are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Leidsplein are 52.3642355, 4.882745556059698.


In [4]:
# create map of Amsterdam using latitude and longitude values
map_ams = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough in zip(ams['Latitude'], ams['Longitude'], ams['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ams)  
    
map_ams

### Let's connect to the Foursquare API and explore the neighborhoods

In [5]:
CLIENT_ID = 'PLJTCRSXWIGFNZWIE5XGDSEYL5CO3VIDKG3RN01YLUQIG0KN' # your Foursquare ID
CLIENT_SECRET = '53WTTCZKTMOOQP1DHBDPEIR4MM0VP1ZXADM22GE4A4YCHX4A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print("Check!")

Check!


In [6]:
# Try the api with the first neighborhood in our df
ams.loc[0,'Borough']

'Bos en Lommer'

In [7]:
b_lat = ams.loc[0, 'Latitude'] # Borough latitude value
b_lng = ams.loc[0, 'Longitude'] # Borough longitude value

b_name = ams.loc[0, 'Borough'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(b_name, 
                                                               b_lat, 
                                                               b_lng))

Latitude and longitude values of Bos en Lommer are 52.380688, 4.855018.


#### Now, let's get the top 100 venues that are in Bos en Lommer within a radius of 1 km.

In [8]:
import requests
# Set parameters and define url
limit = 250 # This doens't seem to work. Results still show only max 100 per borough.
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    b_lat, 
    b_lng, 
    radius, 
    limit)

# Send GET request and inspect results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '605362b44db7b33bb5e1c89d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Stadsdeel West',
  'headerFullLocation': 'Stadsdeel West, Amsterdam',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 35,
  'suggestedBounds': {'ne': {'lat': 52.38968800900001,
    'lng': 4.869734621481298},
   'sw': {'lat': 52.37168799099999, 'lng': 4.840301378518703}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5b51f6f210b3030024ee74ab',
       'name': 'Friedhats FUKU cafe',
       'location': {'address': 'Bos en Lommerweg 136',
        'lat': 52.381161369317034,
        'lng': 4.8552475021908865,
        'labeledLatLngs': [{'label': 'display',
  

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()

35 venues were returned by Foursquare.


,name,categories,lat,lng
0,Friedhats FUKU cafe,Coffee Shop,52.381161,4.855248
1,Samen - Lunch & Juice,Café,52.382616,4.855640
2,Restaurant Betawi,Indonesian Restaurant,52.380984,4.854042
3,Hartje Bos,Sandwich Place,52.379029,4.856613
4,Kratiam Thai,Thai Restaurant,52.380021,4.853776


In [11]:
def getNearbyVenues(boroughs, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, lat, lng in zip(boroughs, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
ams_venues = getNearbyVenues(boroughs = ams['Borough'], 
                             latitudes = ams['Latitude'],
                             longitudes = ams['Longitude']
                                  )

In [13]:
ams_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bos en Lommer,52.380688,4.855018,Friedhats FUKU cafe,52.381161,4.855248,Coffee Shop
1,Bos en Lommer,52.380688,4.855018,Samen - Lunch & Juice,52.382616,4.855640,Café
2,Bos en Lommer,52.380688,4.855018,Restaurant Betawi,52.380984,4.854042,Indonesian Restaurant
3,Bos en Lommer,52.380688,4.855018,Hartje Bos,52.379029,4.856613,Sandwich Place
4,Bos en Lommer,52.380688,4.855018,Kratiam Thai,52.380021,4.853776,Thai Restaurant


In [14]:
ams_venues.shape

(1052, 7)

### Let's recode the results into a few category groups

In [15]:
# ams_venues.to_csv(r'export_ams_venues.csv') # export to csv inspect the results

In [16]:
# Make lists of search strings for each category group
eating = {
    'bagel',
    'bar',
    'beer',
    'bistro',
    'brewery',
    'cafe',
    'club',
    'coffee',
    'deli',
    'diner',
    'food',
    'friterie',
    'joint',
    'place',
    'pub',
    'restaurant',
    'spot',
    'steakhouse',
    'tea'}

shopping = {
    'boutique',
    'grocery',
    'market',
    'pharmacy',
    'shop',
    'store'}

sightseeing = {
    'art',
    'canal',
    'church',
    'comedy',
    'concert',
    'entertainment',
    'event space',
    'farm',
    'fountain',
    'garden',
    'historic',
    'marijuana',
    'monument',
    'museum',
    'music',
    'plaza',
    'pool',
    'road',
    'spa',
    'studio',
    'theater',
    'zoo'}

sleeping = {'bed','hostel','hotel'}

In [18]:
# Make new df for grouping
ams_catgroup = ams_venues

# Convert venue categories to utf
# ams_catgroup['venue_cat'] = [x.decode('utf-8') for x in ams_catgroup['Venue Category']]

# Remove accents and lower case
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

    
ams_catgroup['venue_cat'] = ams_catgroup['Venue Category'].apply(remove_accents)
ams_catgroup['venue_cat'] = ams_catgroup['venue_cat'].str.lower()


ams_catgroup

# import unidecode # Deze code werkte eerst wel met een simpele test op 'Café' maar nu error no attribute 'encode'
# ams_catgroup['Venue Category'].str.encode('utf-8')
# ams_catgroup['Venue Category'] = unidecode.unidecode(ams_catgroup['Venue Category']).lower()


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venue_cat
0,Bos en Lommer,52.380688,4.855018,Friedhats FUKU cafe,52.381161,4.855248,Coffee Shop,coffee shop
1,Bos en Lommer,52.380688,4.855018,Samen - Lunch & Juice,52.382616,4.855640,Café,cafe
2,Bos en Lommer,52.380688,4.855018,Restaurant Betawi,52.380984,4.854042,Indonesian Restaurant,indonesian restaurant
3,Bos en Lommer,52.380688,4.855018,Hartje Bos,52.379029,4.856613,Sandwich Place,sandwich place
4,Bos en Lommer,52.380688,4.855018,Kratiam Thai,52.380021,4.853776,Thai Restaurant,thai restaurant
...,...,...,...,...,...,...,...,...
1047,Grachtengordel West,52.377485,4.890095,Jacketz,52.378229,4.895425,Restaurant,restaurant
1048,Grachtengordel West,52.377485,4.890095,Poké Perfect,52.378642,4.895091,Hawaiian Restaurant,hawaiian restaurant
1049,Grachtengordel West,52.377485,4.890095,Toscanini,52.380145,4.885373,Italian Restaurant,italian restaurant
1050,Grachtengordel West,52.377485,4.890095,Bed & Breakfast Barangay,52.380199,4.893448,Bed & Breakfast,bed & breakfast


In [19]:
def catgroup(cat):
    for cat in ams_catgroup['venue_cat']:
        if any(s in cat for s in sleeping):
            return 'sleeping'
        elif any(s in cat for s in shopping):
            return 'shopping'
        elif any(s in cat for s in eating):
            return 'eating'
        elif any(s in cat for s in sightseeing):
            return 'sightseeing'
        else:
            return 'other'
        

ams_catgroup['cat_group'] = ams_catgroup['venue_cat'].apply(catgroup)

ams_catgroup

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venue_cat,cat_group
0,Bos en Lommer,52.380688,4.855018,Friedhats FUKU cafe,52.381161,4.855248,Coffee Shop,coffee shop,shopping
1,Bos en Lommer,52.380688,4.855018,Samen - Lunch & Juice,52.382616,4.855640,Café,cafe,shopping
2,Bos en Lommer,52.380688,4.855018,Restaurant Betawi,52.380984,4.854042,Indonesian Restaurant,indonesian restaurant,shopping
3,Bos en Lommer,52.380688,4.855018,Hartje Bos,52.379029,4.856613,Sandwich Place,sandwich place,shopping
4,Bos en Lommer,52.380688,4.855018,Kratiam Thai,52.380021,4.853776,Thai Restaurant,thai restaurant,shopping
...,...,...,...,...,...,...,...,...,...
1047,Grachtengordel West,52.377485,4.890095,Jacketz,52.378229,4.895425,Restaurant,restaurant,shopping
1048,Grachtengordel West,52.377485,4.890095,Poké Perfect,52.378642,4.895091,Hawaiian Restaurant,hawaiian restaurant,shopping
1049,Grachtengordel West,52.377485,4.890095,Toscanini,52.380145,4.885373,Italian Restaurant,italian restaurant,shopping
1050,Grachtengordel West,52.377485,4.890095,Bed & Breakfast Barangay,52.380199,4.893448,Bed & Breakfast,bed & breakfast,shopping


In [ ]:
ams_catgroup.drop(['cat_group'], axis=1)

In [ ]:
# Group by cat_group and count
ams_catgroup.groupby('cat_group').count()

In [ ]:
#

























































#

### Let's see how many venues each borough has

In [ ]:
ams_venues.groupby('Borough').count()

In [ ]:
#  And print the number of unique categories
print('There are {} uniques categories.'.format(len(ams_venues['Venue Category'].unique())))

### One hot encoding and frequency

In [ ]:
# one hot encoding
ams_onehot = pd.get_dummies(ams_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
ams_onehot['Borough'] = ams_venues['Borough']

# move neighborhood column to the first column
fixed_columns = [ams_onehot.columns[-1]] + list(ams_onehot.columns[:-1])
ams_onehot = ams_onehot[fixed_columns]
cols_to_move = ['Borough']
ams_onehot = ams_onehot[ cols_to_move + [ col for col in ams_onehot.columns if col not in cols_to_move ] ]

print(ams_onehot.shape)

ams_onehot.head()

In [ ]:
# Now group borough and take the mean to show venue occurrence per borough
ams_grouped = ams_onehot.groupby(['Borough']).mean().reset_index()
ams_grouped

In [ ]:
# Also group and take the sum to show venues per borough
import numpy as np

ams_grouped_sum = ams_onehot.groupby(['Borough']).sum().T
ams_grouped_sum

In [ ]:
# Only select columns containing museum 

museum_cols = [col for col in ams_grouped_sum.index if 'Museum' in col]
print(list(ams_grouped_sum.index))
print(museum_cols) 

# Now continue grouping with count of museum type per borough


In [ ]:
# Let's count the unique venues per borough
ams_venues_unique = ams_venues.groupby('Borough')['Venue Category'].nunique()
ams_venues_unique

### Show the 5 most popular types of venues per borough

In [ ]:
# Let's count the venues per borough
ams_venues_total = ams_venues.groupby('Borough')['Venue Category'].count()
ams_venues_total.to_frame()
ams_venues_total

In [ ]:
# And combine into one overview --- Weet echt even niet hoe dit nou moet...
# ams_counts = ams_venues_unique.to_frame()
# ams_counts = ams_counts.join(ams_venues_total.set_index('Borough'), on = 'Borough')
# ams_venues_unique['Borough'] = ams_venues_unique.index
# ams_venues_unique
# ams_counts = pd.DataFrame(list(zip(ams_venues_unique, ams_venues_total)), columns = ['Unique', 'Total'])
# ams_counts

In [ ]:
num_top_venues = 5

# ams_grouped = ams_grouped.drop(['Borough'], axis=1)

for hood in ams_grouped['Borough']:
    print("----"+hood+"----")
    temp = ams_grouped[ams_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Let's summarize the most popular venue info per postal code into one table

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Borough'] = ams_grouped['Borough']

for ind in np.arange(ams_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(ams_grouped.iloc[ind, :], num_top_venues)

venues_sorted

### Clustering analysis

Run _k_-means to cluster the neighborhood into a few clusters.

In [ ]:
# set number of clusters
kclusters = 5

ams_cluster = ams_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ams_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
ams_cluster = venues_sorted
ams_cluster = ams_cluster.insert(0, 'Cluster Labels', kmeans.labels_)


In [ ]:
ams_cluster

In [ ]:
# merge grouped with geo data to add latitude/longitude for each neighborhood
ams_merged = ams

ams_merged = ams_merged.join(venues_sorted.set_index('Borough'), on='Borough', how='right')

ams_merged # check the last columns!


In [ ]:
ams_merged = ams_merged.drop(ams_merged.columns[3:5], axis=1)
# ams_merged = ams_merged.drop(ams_merged.columns[3], axis=1)
ams_merged

### Time to visualize the results!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ams_merged['Latitude'], ams_merged['Longitude'], ams_merged['Borough'], ams_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's observe the clusters in more detail one by one

In [ ]:
# Cluster 1 - Most common mix of residential with lots of hospitality and variety
ams_merged.loc[ams_merged['Cluster Labels'] == 0, ams_merged.columns[[0] + list(range(4, ams_merged.shape[1]))]]

In [ ]:
# Cluster 2 - Residential
ams_merged.loc[ams_merged['Cluster Labels'] == 1, ams_merged.columns[[0] + list(range(4, ams_merged.shape[1]))]]

In [ ]:
# Cluster 3 - Zoo area
ams_merged.loc[ams_merged['Cluster Labels'] == 2, ams_merged.columns[[0] + list(range(4, ams_merged.shape[1]))]]

In [ ]:
# Cluster 4 - Somehow different from cluster 1 (boring?)
ams_merged.loc[ams_merged['Cluster Labels'] == 3, ams_merged.columns[[0] + list(range(4, ams_merged.shape[1]))]]

In [ ]:
# Cluster 5 - Touristic with musea and ganja
ams_merged.loc[ams_merged['Cluster Labels'] == 4, ams_merged.columns[[0] + list(range(4, ams_merged.shape[1]))]]

In [ ]:
ams_venues_total

In [ ]:
ams_grouped['Hotel']

In [ ]:
# ams_merged.drop('Hotel', axis=1, inplace=True)
# ams_merged['Hotel'] = ams_grouped['Hotel']
ams_merged = pd.merge(ams_merged, ams_grouped[['Borough','Hotel']], on='Borough', how = 'left')
ams_merged # check the last columns!

In [ ]:
cols_to_move = ['Borough','Latitude','Longitude','Hotel']
ams_merged = ams_merged[ cols_to_move + [ col for col in ams_merged.columns if col not in cols_to_move ] ]

In [ ]:
ams_merged['Hotel'] = ams_merged['Hotel']*100 
ams_merged